# Preliminary Data and Functions

In [1]:
import numpy as np
import re
from random import randint

#Importing Data
wordsList = np.load('Numpy Arrays/wordsList.npy')
wordsList = wordsList.tolist()
wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('Numpy Arrays/wordVectors.npy')
#text = np.load('Numpy Arrays/text.npy')
the_labels = np.load('Numpy Arrays/labels.npy')
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

#Helper Functions
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())
ids = np.load('Numpy Arrays/idsMatrix.npy')
def getTrainBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1, 1000000)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label
def getTestBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1000001, the_labels.shape[0]-1)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label

# BRNN Model

### Hyper Parameters

In [2]:
batchSize = 512
lstmUnits = 128
numClasses = 2
iterations = 10000
learning_rate = 0.001
maxSeqLength = 20

In [3]:
import tensorflow as tf
with tf.name_scope("Input_Data") as scope:
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name='Sentiment_Labels')
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name='Text')

with tf.name_scope("Data_Preprocessing") as scope:
    data = tf.Variable(tf.zeros([batchSize, maxSeqLength, 3]),dtype=tf.float32, name='Embed_Text')
    data = tf.nn.embedding_lookup(wordVectors,input_data)

In [4]:
with tf.name_scope("MultiBRNN") as scope:
    cell = tf.nn.rnn_cell.LSTMCell(lstmUnits)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.9)
    cell1 = tf.nn.rnn_cell.LSTMCell(lstmUnits)
    cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.9)

    (forward_output, backward_output), _ = \
        tf.nn.bidirectional_dynamic_rnn(cell, cell1, inputs=data, dtype=tf.float32,scope='BLSTM_1')
    outputs = tf.concat([forward_output, backward_output], axis=2)

    #Second BLSTM using the output of previous layer as an input.
    cell2 = tf.nn.rnn_cell.LSTMCell(lstmUnits)
    cell2 = tf.nn.rnn_cell.DropoutWrapper(cell2, output_keep_prob=0.9)
    cell3 = tf.nn.rnn_cell.LSTMCell(lstmUnits)
    cell3 = tf.nn.rnn_cell.DropoutWrapper(cell3, output_keep_prob=0.9)

    (forward_output, backward_output), _ = \
        tf.nn.bidirectional_dynamic_rnn(cell2, cell3, inputs=outputs, dtype=tf.float32,scope='BLSTM_2')
    outputs = tf.concat([forward_output, backward_output], axis=2)

In [5]:
with tf.name_scope("Prediction_Parameters") as scope:
    weight = tf.Variable(tf.truncated_normal([2*lstmUnits, numClasses]), name='Weights')
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name='bias')
    value = tf.transpose(outputs, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
with tf.name_scope("Prediction") as scope:
    prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [6]:
with tf.name_scope("Accuracy") as scope:
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


In [7]:
with tf.name_scope("Loss") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
with tf.name_scope("Train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
import datetime
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
validation = tf.summary.scalar('Validation', accuracy)
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [8]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


# Training

In [9]:
for i in range(iterations+1):
        #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
    #Write summary to Tensorboard
#    if (i % 5 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)
#        nextTestBatch, nextTestBatchLabels = getTestBatch();
#        val = sess.run(validation, {input_data: nextTestBatch, labels: nextTestBatchLabels})
#        writer.add_summary(val, i)
    if (i%500 == 0):
        print("Loss: " + str(sess.run(loss, {input_data: nextBatch, labels: nextBatchLabels})))
save_path = saver.save(sess, "BRNN2/brnn.ckpt", global_step=i)
writer.close()

Loss: 0.7279986
Loss: 0.57561207
Loss: 0.61590827
Loss: 0.5531422
Loss: 0.54807734
Loss: 0.56958795
Loss: 0.55133176
Loss: 0.5434594
Loss: 0.5497587
Loss: 0.52215225
Loss: 0.54473805
Loss: 0.546203
Loss: 0.5315193
Loss: 0.49219894
Loss: 0.5164892
Loss: 0.50491595
Loss: 0.5310552
Loss: 0.5056318
Loss: 0.52156454
Loss: 0.5259479
Loss: 0.5155761


NameError: name 'writer' is not defined

In [10]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix
def test(sent):
    x = ['negetive', 'positive']
    sent = getSentenceMatrix(sent)
    pred = sess.run(prediction, {input_data:sent})[0]
    return(pred)
def results():
    arr = np.array([[1, 0]], dtype=np.float32)
    arr_pred = np.array([test("my life sucks")])
    for i in range(512, ids.shape[0], 512):
        arr = np.vstack((arr, the_labels[i-512:i]))
        arr_pred = np.vstack((arr_pred, sess.run(prediction, {input_data:ids[i-512:i]})))
    x = np.argmax(arr_pred, axis=1)
    y = np.argmax(arr, axis=1)
    return((np.sum(np.equal(x, y))/x.shape))

In [11]:
results()

array([0.77661345])

In [12]:
def find_time():
    import time
    cx = np.random.randint(513, ids.shape[0])
    start_time = time.time()
    sess.run(prediction, {input_data:ids[cx-512:cx]})
#    print("---| %s seconds ---" % (time.time() - start_time))
    return(time.time() - start_time)
find_time()
def find():
    temp = 0
    for i in range(10):
        temp+=find_time()
    print(temp/10)
find()

0.6899475812911987
